In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids = None, sample_size = 100):
        word_ids = [start_id]
        
        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())
            
            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
                
        return word_ids

In [9]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

#시작 문자와 건너뜀 문자 설정
model = RnnlmGen()
model.load_params('./ch06/Rnnlm.pkl')

start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>','$']
skip_ids = [word_to_id[w] for w in skip_words]

#문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you see jobs.
 we prefer the payroll and for their own risks and eventually.
 there are buy cuts that are buying it also are buying confidence according to the situation says steven johnson a metropolitan analyst.
 fusion d. stone the arizona son former 's vice chairman ralph p. resigned and the chairman of the banking firm in the ciba-geigy resort.
 he takes a blow to bolster its device with mr. honecker.
 for explosions mr. laband said bonds that use of reasons said frank weiss the american jose bank chief of the leading business do not


In [4]:
' '.join(['you', 'say', 'goodbye'])

'you say goodbye'

In [17]:
from dataset import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data('../dataset/addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

print(x_train[0])
print(t_train[0])

print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)
[ 3  0  2  0  0 11  5]
[ 6  0 11  7  5]
71+118 
_189 


In [20]:
class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size #하이퍼 파라미터
        rn = np.random.randn #난수값
        
        embed_W = (rn(V, D) / 100).astype('f')              #임베딩 벡터 가중치 
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')   #lstm 가중치
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')   #히든레이어
        lstm_b = np.zeros(4 * H).astype('f')                #
        
        self.embed = TimeEmbedding(embed_W) #임베딩 벡터를 삽입한다
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful = False)
        
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None
        
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        self.hs = hs
        return hs[:, -1, :]
    
    def backward(self, dh):
        dhs = np.zeros_like(dhs)
        dhs[:, -1,:] = dh
        
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout

In [22]:
class Decoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)
        self.params, self.grads = [], []
        
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, xs, h):
        self.lstm.set_state(h)
        
        out = self.embed.forward(xs)
        out = self.lstm.forward(out)
        score = self.affine.forward(out)
        return score
    
    def backward(self, dscore):
        dout = self.affine.backward(dscore)
        dout = self.lstm.backward(dout)
        dout = self.embed.backward(dout)
        dh = self.lstm.dh
        return dh
    
    def generate(self, h, start_id, sample_size):
        sampled = []
        sample_id = start_id
        self.lstm.set_state(h)
        
        for _ in range(sample_size):
            x = np.array(sample_id).reshape((1, 1))
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)
            
            sample_id = np.argmax(score.flatten())
            sampled.append(int(sample_id))
        return sampled

In [24]:
from common.base_model import BaseModel


class Seq2Seq(BaseModel):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        self.encoder = Encoder(V, D, H)
        self.decoder = Decoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()
        
        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads
        
    def forward(self, xs, ts):
        decoder_xs, decoder_ts = ts[:, :-1], ts[:, 1:]
        h = self.encoder.forward(xs)
        score = self.decoder.forward(decoder_xs, h)
        loss = self.softmax.forward(score, decoder_ts)
        
    def backward(self, dout=1):
        dout = self.softmax.backward(dout)
        dh = self.decoder.backward(dout)
        dout = self.encoder.backward(dh)
        return dout
    
    def generate(self, xs, start_id, sample_size):
        h = self.encoder.forward(xs)
        sampled = self.decoder.generate(h, start_id, sample_size)
        return sampled

In [27]:
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

#하이퍼 파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

#모델 / 옵티마이저 / 트레이너 생성
model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1, batch_size = batch_size, max_grad=max_grad)
    correct_num = 0
    
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model,question, correct, id_to_char, verbose)
        
    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 2.56
| 에폭 1 |  반복 21 / 351 | 시간 0[s] | 손실 2.53
| 에폭 1 |  반복 41 / 351 | 시간 1[s] | 손실 2.17
| 에폭 1 |  반복 61 / 351 | 시간 1[s] | 손실 1.96
| 에폭 1 |  반복 81 / 351 | 시간 2[s] | 손실 1.92
| 에폭 1 |  반복 101 / 351 | 시간 3[s] | 손실 1.87
| 에폭 1 |  반복 121 / 351 | 시간 3[s] | 손실 1.85
| 에폭 1 |  반복 141 / 351 | 시간 4[s] | 손실 1.83
| 에폭 1 |  반복 161 / 351 | 시간 4[s] | 손실 1.79
| 에폭 1 |  반복 181 / 351 | 시간 5[s] | 손실 1.77
| 에폭 1 |  반복 201 / 351 | 시간 6[s] | 손실 1.77
| 에폭 1 |  반복 221 / 351 | 시간 6[s] | 손실 1.76
| 에폭 1 |  반복 241 / 351 | 시간 7[s] | 손실 1.76
| 에폭 1 |  반복 261 / 351 | 시간 8[s] | 손실 1.76
| 에폭 1 |  반복 281 / 351 | 시간 8[s] | 손실 1.75
| 에폭 1 |  반복 301 / 351 | 시간 9[s] | 손실 1.74
| 에폭 1 |  반복 321 / 351 | 시간 9[s] | 손실 1.75
| 에폭 1 |  반복 341 / 351 | 시간 10[s] | 손실 1.74
Q 77+85  
T 162 
X 100 
---
Q 975+164
T 1139
X 1000
---
Q 582+84 
T 666 
X 1000
---
Q 8+155  
T 163 
X 100 
---
Q 367+55 
T 422 
X 1000
---
Q 600+257
T 857 
X 1000
---
Q 761+292
T 1053
X 1000
---
Q 830+597
T 1427
X 1000
---
Q 26+83

Q 77+85  
T 162 
X 166 
---
Q 975+164
T 1139
X 1166
---
Q 582+84 
T 666 
X 676 
---
Q 8+155  
T 163 
X 168 
---
Q 367+55 
T 422 
X 430 
---
Q 600+257
T 857 
X 873 
---
Q 761+292
T 1053
X 1073
---
Q 830+597
T 1427
X 1441
---
Q 26+838 
T 864 
X 871 
---
Q 143+93 
T 236 
X 227 
---
검증 정확도 3.440%
| 에폭 9 |  반복 1 / 351 | 시간 0[s] | 손실 1.02
| 에폭 9 |  반복 21 / 351 | 시간 0[s] | 손실 1.01
| 에폭 9 |  반복 41 / 351 | 시간 1[s] | 손실 1.00
| 에폭 9 |  반복 61 / 351 | 시간 2[s] | 손실 1.00
| 에폭 9 |  반복 81 / 351 | 시간 2[s] | 손실 0.99
| 에폭 9 |  반복 101 / 351 | 시간 3[s] | 손실 0.99
| 에폭 9 |  반복 121 / 351 | 시간 4[s] | 손실 1.00
| 에폭 9 |  반복 141 / 351 | 시간 4[s] | 손실 0.99
| 에폭 9 |  반복 161 / 351 | 시간 5[s] | 손실 0.99
| 에폭 9 |  반복 181 / 351 | 시간 6[s] | 손실 0.99
| 에폭 9 |  반복 201 / 351 | 시간 6[s] | 손실 0.99
| 에폭 9 |  반복 221 / 351 | 시간 7[s] | 손실 0.99
| 에폭 9 |  반복 241 / 351 | 시간 7[s] | 손실 1.04
| 에폭 9 |  반복 261 / 351 | 시간 8[s] | 손실 1.03
| 에폭 9 |  반복 281 / 351 | 시간 9[s] | 손실 1.01
| 에폭 9 |  반복 301 / 351 | 시간 9[s] | 손실 1.01
| 에폭 9 |  반복 321 / 351 |

| 에폭 16 |  반복 161 / 351 | 시간 5[s] | 손실 0.95
| 에폭 16 |  반복 181 / 351 | 시간 5[s] | 손실 0.95
| 에폭 16 |  반복 201 / 351 | 시간 6[s] | 손실 0.95
| 에폭 16 |  반복 221 / 351 | 시간 7[s] | 손실 0.94
| 에폭 16 |  반복 241 / 351 | 시간 7[s] | 손실 0.94
| 에폭 16 |  반복 261 / 351 | 시간 8[s] | 손실 0.94
| 에폭 16 |  반복 281 / 351 | 시간 8[s] | 손실 0.93
| 에폭 16 |  반복 301 / 351 | 시간 9[s] | 손실 0.89
| 에폭 16 |  반복 321 / 351 | 시간 10[s] | 손실 0.84
| 에폭 16 |  반복 341 / 351 | 시간 10[s] | 손실 0.84
Q 77+85  
T 162 
X 161 
---
Q 975+164
T 1139
X 1107
---
Q 582+84 
T 666 
X 662 
---
Q 8+155  
T 163 
X 172 
---
Q 367+55 
T 422 
X 419 
---
Q 600+257
T 857 
X 862 
---
Q 761+292
T 1053
X 1039
---
Q 830+597
T 1427
X 1404
---
Q 26+838 
T 864 
X 869 
---
Q 143+93 
T 236 
X 239 
---
검증 정확도 7.040%
| 에폭 17 |  반복 1 / 351 | 시간 0[s] | 손실 0.86
| 에폭 17 |  반복 21 / 351 | 시간 0[s] | 손실 0.85
| 에폭 17 |  반복 41 / 351 | 시간 1[s] | 손실 0.86
| 에폭 17 |  반복 61 / 351 | 시간 1[s] | 손실 0.85
| 에폭 17 |  반복 81 / 351 | 시간 2[s] | 손실 0.84
| 에폭 17 |  반복 101 / 351 | 시간 3[s] | 손실 0.87
| 에폭 1

검증 정확도 9.300%
| 에폭 24 |  반복 1 / 351 | 시간 0[s] | 손실 0.75
| 에폭 24 |  반복 21 / 351 | 시간 0[s] | 손실 0.79
| 에폭 24 |  반복 41 / 351 | 시간 1[s] | 손실 0.79
| 에폭 24 |  반복 61 / 351 | 시간 1[s] | 손실 0.79
| 에폭 24 |  반복 81 / 351 | 시간 2[s] | 손실 0.77
| 에폭 24 |  반복 101 / 351 | 시간 3[s] | 손실 0.77
| 에폭 24 |  반복 121 / 351 | 시간 3[s] | 손실 0.76
| 에폭 24 |  반복 141 / 351 | 시간 4[s] | 손실 0.77
| 에폭 24 |  반복 161 / 351 | 시간 4[s] | 손실 0.80
| 에폭 24 |  반복 181 / 351 | 시간 5[s] | 손실 0.80
| 에폭 24 |  반복 201 / 351 | 시간 6[s] | 손실 0.78
| 에폭 24 |  반복 221 / 351 | 시간 6[s] | 손실 0.77
| 에폭 24 |  반복 241 / 351 | 시간 7[s] | 손실 0.77
| 에폭 24 |  반복 261 / 351 | 시간 8[s] | 손실 0.77
| 에폭 24 |  반복 281 / 351 | 시간 8[s] | 손실 0.78
| 에폭 24 |  반복 301 / 351 | 시간 9[s] | 손실 0.76
| 에폭 24 |  반복 321 / 351 | 시간 9[s] | 손실 0.76
| 에폭 24 |  반복 341 / 351 | 시간 10[s] | 손실 0.77
Q 77+85  
T 162 
X 161 
---
Q 975+164
T 1139
X 1137
---
Q 582+84 
T 666 
X 662 
---
Q 8+155  
T 163 
X 162 
---
Q 367+55 
T 422 
X 419 
---
Q 600+257
T 857 
X 846 
---
Q 761+292
T 1053
X 1039
---
Q 8

In [1]:
from seq2seq import Seq2seq, Encoder 

class PeekySeq2seq(Seq2seq):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        self.encoder = Encoder(V, D, H)
        self.decoder = PeekyDecoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()
        
        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads
        

NameError: name 'vocab_size' is not defined

In [3]:
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

#하이퍼 파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

#모델 / 옵티마이저 / 트레이너 생성
#model = Seq2seq(vocab_size, wordvec_size, hidden_size)
model = PeekySeq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1, batch_size = batch_size, max_grad=max_grad)
    correct_num = 0
    
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model,question, correct, id_to_char, verbose)
        
    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 2.57
| 에폭 1 |  반복 21 / 351 | 시간 0[s] | 손실 2.49
| 에폭 1 |  반복 41 / 351 | 시간 1[s] | 손실 2.20
| 에폭 1 |  반복 61 / 351 | 시간 1[s] | 손실 1.96
| 에폭 1 |  반복 81 / 351 | 시간 2[s] | 손실 1.84
| 에폭 1 |  반복 101 / 351 | 시간 3[s] | 손실 1.80
| 에폭 1 |  반복 121 / 351 | 시간 3[s] | 손실 1.79
| 에폭 1 |  반복 141 / 351 | 시간 4[s] | 손실 1.77
| 에폭 1 |  반복 161 / 351 | 시간 5[s] | 손실 1.77
| 에폭 1 |  반복 181 / 351 | 시간 5[s] | 손실 1.76
| 에폭 1 |  반복 201 / 351 | 시간 6[s] | 손실 1.76
| 에폭 1 |  반복 221 / 351 | 시간 7[s] | 손실 1.75
| 에폭 1 |  반복 241 / 351 | 시간 7[s] | 손실 1.76
| 에폭 1 |  반복 261 / 351 | 시간 8[s] | 손실 1.75
| 에폭 1 |  반복 281 / 351 | 시간 8[s] | 손실 1.74
| 에폭 1 |  반복 301 / 351 | 시간 9[s] | 손실 1.73
| 에폭 1 |  반복 321 / 351 | 시간 10[s] | 손실 1.73
| 에폭 1 |  반복 341 / 351 | 시간 10[s] | 손실 1.73
Q 77+85  
T 162 
X 107 
---
Q 975+164
T 1139
X 1011
---
Q 582+84 
T 666 
X 103 
---
Q 8+155  
T 163 
X 101 
---
Q 367+55 
T 422 
X 103 
---
Q 600+257
T 857 
X 1023
---
Q 761+292
T 1053
X 1023
---
Q 830+597
T 1427
X 1011
---
Q 26+8

Q 77+85  
T 162 
X 164 
---
Q 975+164
T 1139
X 1137
---
Q 582+84 
T 666 
X 665 
---
Q 8+155  
T 163 
X 160 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
X 849 
---
Q 761+292
T 1053
X 1037
---
Q 830+597
T 1427
X 1437
---
Q 26+838 
T 864 
X 869 
---
Q 143+93 
T 236 
X 231 
---
검증 정확도 12.480%
| 에폭 9 |  반복 1 / 351 | 시간 0[s] | 손실 0.70
| 에폭 9 |  반복 21 / 351 | 시간 0[s] | 손실 0.70
| 에폭 9 |  반복 41 / 351 | 시간 1[s] | 손실 0.70
| 에폭 9 |  반복 61 / 351 | 시간 2[s] | 손실 0.70
| 에폭 9 |  반복 81 / 351 | 시간 2[s] | 손실 0.70
| 에폭 9 |  반복 101 / 351 | 시간 3[s] | 손실 0.69
| 에폭 9 |  반복 121 / 351 | 시간 4[s] | 손실 0.69
| 에폭 9 |  반복 141 / 351 | 시간 5[s] | 손실 0.68
| 에폭 9 |  반복 161 / 351 | 시간 5[s] | 손실 0.68
| 에폭 9 |  반복 181 / 351 | 시간 6[s] | 손실 0.67
| 에폭 9 |  반복 201 / 351 | 시간 7[s] | 손실 0.68
| 에폭 9 |  반복 221 / 351 | 시간 7[s] | 손실 0.67
| 에폭 9 |  반복 241 / 351 | 시간 8[s] | 손실 0.67
| 에폭 9 |  반복 261 / 351 | 시간 9[s] | 손실 0.67
| 에폭 9 |  반복 281 / 351 | 시간 9[s] | 손실 0.66
| 에폭 9 |  반복 301 / 351 | 시간 10[s] | 손실 0.66
| 에폭 9 |  반복 321 / 351

| 에폭 16 |  반복 161 / 351 | 시간 6[s] | 손실 0.43
| 에폭 16 |  반복 181 / 351 | 시간 6[s] | 손실 0.43
| 에폭 16 |  반복 201 / 351 | 시간 7[s] | 손실 0.42
| 에폭 16 |  반복 221 / 351 | 시간 8[s] | 손실 0.43
| 에폭 16 |  반복 241 / 351 | 시간 8[s] | 손실 0.44
| 에폭 16 |  반복 261 / 351 | 시간 9[s] | 손실 0.43
| 에폭 16 |  반복 281 / 351 | 시간 10[s] | 손실 0.42
| 에폭 16 |  반복 301 / 351 | 시간 11[s] | 손실 0.43
| 에폭 16 |  반복 321 / 351 | 시간 11[s] | 손실 0.45
| 에폭 16 |  반복 341 / 351 | 시간 12[s] | 손실 0.44
Q 77+85  
T 162 
X 163 
---
Q 975+164
T 1139
X 1137
---
Q 582+84 
T 666 
X 667 
---
Q 8+155  
T 163 
X 162 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
X 856 
---
Q 761+292
T 1053
X 1048
---
Q 830+597
T 1427
X 1429
---
Q 26+838 
T 864 
O 864 
---
Q 143+93 
T 236 
O 236 
---
검증 정확도 31.180%
| 에폭 17 |  반복 1 / 351 | 시간 0[s] | 손실 0.41
| 에폭 17 |  반복 21 / 351 | 시간 0[s] | 손실 0.41
| 에폭 17 |  반복 41 / 351 | 시간 1[s] | 손실 0.41
| 에폭 17 |  반복 61 / 351 | 시간 2[s] | 손실 0.42
| 에폭 17 |  반복 81 / 351 | 시간 2[s] | 손실 0.42
| 에폭 17 |  반복 101 / 351 | 시간 3[s] | 손실 0.42
| 에

검증 정확도 70.680%
| 에폭 24 |  반복 1 / 351 | 시간 0[s] | 손실 0.18
| 에폭 24 |  반복 21 / 351 | 시간 0[s] | 손실 0.18
| 에폭 24 |  반복 41 / 351 | 시간 1[s] | 손실 0.18
| 에폭 24 |  반복 61 / 351 | 시간 1[s] | 손실 0.18
| 에폭 24 |  반복 81 / 351 | 시간 2[s] | 손실 0.17
| 에폭 24 |  반복 101 / 351 | 시간 3[s] | 손실 0.18
| 에폭 24 |  반복 121 / 351 | 시간 3[s] | 손실 0.17
| 에폭 24 |  반복 141 / 351 | 시간 4[s] | 손실 0.17
| 에폭 24 |  반복 161 / 351 | 시간 5[s] | 손실 0.17
| 에폭 24 |  반복 181 / 351 | 시간 5[s] | 손실 0.18
| 에폭 24 |  반복 201 / 351 | 시간 6[s] | 손실 0.18
| 에폭 24 |  반복 221 / 351 | 시간 7[s] | 손실 0.17
| 에폭 24 |  반복 241 / 351 | 시간 7[s] | 손실 0.18
| 에폭 24 |  반복 261 / 351 | 시간 8[s] | 손실 0.18
| 에폭 24 |  반복 281 / 351 | 시간 9[s] | 손실 0.17
| 에폭 24 |  반복 301 / 351 | 시간 9[s] | 손실 0.17
| 에폭 24 |  반복 321 / 351 | 시간 10[s] | 손실 0.16
| 에폭 24 |  반복 341 / 351 | 시간 11[s] | 손실 0.16
Q 77+85  
T 162 
O 162 
---
Q 975+164
T 1139
O 1139
---
Q 582+84 
T 666 
O 666 
---
Q 8+155  
T 163 
X 164 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
O 857 
---
Q 761+292
T 1053
X 1054
---
Q